# Now use compare_tworuns from myutils.py to compare values from two DataFrames with Camera info

In [1]:
import sys
sys.path.append('/home/s/seanmacb/Camera/pkgs')
import myutils_ar.myutils as myu
import run_six_utils as rsu
import eo_pipe
import matplotlib.pyplot as plt
import pandas as pd
import lsst.daf.butler as daf_butler
from lsst.obs.lsst import LsstCam, LsstTS8
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import lsst.geom
from lsst.afw import cameraGeom
from lsst.obs.lsst import LsstCam
import pandas as pd

In [2]:
def get_mins(df1,df2,label="BRIGHT_COLUMNS"):
    return np.min([np.array(df1[label]),np.array(df2[label])])

def get_max(df1,df2,label="BRIGHT_COLUMNS"):
    return np.max([np.array(df1[label]),np.array(df2[label])])

In [3]:
repo = '/repo/ir2'
operator = 'lsstccs'
acq_runs = ['13401','13550']  # Run 6, run 6b.
weeklys = ['w_2023_24','w_2023_41']

In [1]:
df_dict = {}
butler = daf_butler.Butler(repo)
for acq_run,weekly in zip(acq_runs,weeklys):
    # acq_run,weekly = acq_runs[run_iter],weeklys[run_iter]
    collection = butler.registry.queryCollections(f"u/{operator}/eo_*_{acq_run}_{weekly}",
                                                   collectionTypes=daf_butler.CollectionType.CHAINED)
    df_dict[int(acq_run)] = myu.eopipe_DictToDfz(eo_pipe.get_amp_data(repo,collection)) # For each run
# myu.compare_tworuns(df1,df2,run1,run2,minxy,maxxy,quantity='READ_NOISE',draw_line=True,legend_loc='lower right',scale='linear',save=None,fname=None) # once in each dataframe

NameError: name 'daf_butler' is not defined

In [57]:
df_dict[13401][["DARK_CURRENT_95","DARK_CURRENT_MEDIAN"]]

,DARK_CURRENT_95,DARK_CURRENT_MEDIAN
0,0.029763,0.007721
1,0.029854,0.008165
2,0.029877,0.008227
3,0.029731,0.008100
4,0.029503,0.007850
...,...,...
3211,0.028658,0.007796
3212,0.029082,0.007469
3213,0.028391,0.007423
3214,0.027719,0.006973


In [62]:
butler.registry.queryCollections(f"u/{operator}/*{acq_run}_{weekly}",
                                                   collectionTypes=daf_butler.CollectionType.CHAINED)

['u/lsstccs/bias_13550_w_2023_41',
 'u/lsstccs/dark_13550_w_2023_41',
 'u/lsstccs/flat_13550_w_2023_41',
 'u/lsstccs/defects_13550_w_2023_41',
 'u/lsstccs/ptc_13550_w_2023_41',
 'u/lsstccs/eo_read_noise_13550_w_2023_41',
 'u/lsstccs/eo_dark_current_13550_w_2023_41',
 'u/lsstccs/eo_raft_amp_correlations_13550_w_2023_41',
 'u/lsstccs/eo_eper_13550_w_2023_41',
 'u/lsstccs/eo_raft_calib_mosaics_13550_w_2023_41',
 'u/lsstccs/eo_cti_vs_flux_13550_w_2023_41',
 'u/lsstccs/eo_ptc_plots_13550_w_2023_41',
 'u/lsstccs/eo_linearity_plots_13550_w_2023_41',
 'u/lsstccs/eo_divisadero_tearing_13550_w_2023_41',
 'u/lsstccs/eo_bf_analysis_13550_w_2023_41',
 'u/lsstccs/eo_bias_shifts_13550_w_2023_41',
 'u/lsstccs/eo_bright_defects_13550_w_2023_41',
 'u/lsstccs/eo_persistence_13550_w_2023_41',
 'u/lsstccs/eo_dark_defects_13550_w_2023_41',
 'u/lsstccs/eo_bias_stability_13550_w_2023_41',
 'u/lsstccs/eo_raft_lambda_mosaics_13550_w_2023_41']

In [5]:
# df_dict[0].columns.values

In [5]:
quantities = ['BRIGHT_COLUMNS', 'BRIGHT_PIXELS','DARK_COLUMNS', 'DARK_PIXELS']
scale = ['linear','linear','linear','log']
# quantities = ['BRIGHT_PIXELS', 'DARK_PIXELS']
iter=0
ids=['','b']
minx = [-2,1100]
maxx = [30,2500]
for quantity in quantities:
    myu.compare_tworuns(df_dict[0],df_dict[1],minx[iter],maxx[iter],"6 ({var})".format(var=acq_runs[0]),
                        "6b ({var})".format(var=acq_runs[1]),quantity=quantity,save=True,
                        fname='/home/s/seanmacb/Run6Comparisons/Figures/compare_tworuns/comparison_wide_'+quantity+".pdf",
                        graphscale=0.2)
    iter+=1

# Immediate todo: fix axes issue and scaling

NameError: name 'df_dict' is not defined

## Take the defects in `df_dict`, and take difference of the same amps along defect columns

In [8]:
# Use this slice for now since the dark defects are complicated by the sensor edge effects
diff_columns = ['BRIGHT_COLUMNS', 'BRIGHT_PIXELS','DARK_COLUMNS', 'DARK_PIXELS'][0:2] 

pix_threshold, column_threshold = 10,0 # Threshold in pixels, columns for a difference between EO runs to be flagged
thresholds = [column_threshold,pix_threshold]
conditions = {} # Change to dict for easier data handling
for col,threshold in zip(diff_columns,thresholds):
    conditions[col] = threshold

In [9]:
df1, df2 = df_dict[13550],df_dict[13401]

In [12]:
rsu.getDiffDF(df1,df2)

,BAY_SLOT,SEGMENT,BAY_SLOT_SEGMENT,BRIGHT_COLUMNS_DIFFERENCE,BRIGHT_PIXELS_DIFFERENCE
36,R00_SW0,C14,R00_SW0_C14,1,12
52,R01_S00,C14,R01_S00_C14,4,66
59,R01_S00,C04,R01_S00_C04,7,135
60,R01_S00,C03,R01_S00_C03,1,15
98,R01_S10,C12,R01_S10_C12,4,84
161,R01_S21,C11,R01_S21_C11,1,19
482,R04_SG0,C12,R04_SG0_C12,7,55
497,R04_SG1,C11,R04_SG1_C11,5,2559
645,R10_S21,C15,R10_S21_C15,2,21
919,R12_S20,C17,R12_S20_C17,2,1054


## Look in the defect collection to see if there is positional data within the object - there should be!

In [20]:
collection1 = 'u/lsstccs/eo_bright_defects_13550_w_2023_41'
collection2 = 'u/lsstccs/eo_bright_defects_13401_w_2023_24'
collections = 'LSSTCam/raw/all,LSSTCam/calib'.split(",") # Defining collections (that are ignored by below kernel)


In [18]:
rsu.fetchAllRefs(repo,collection__,[13550],"eoBrightDefects",55)

[DatasetRef(DatasetType('eoBrightDefects', {instrument, detector}, Defects), {instrument: 'LSSTCam', detector: 55}, run='u/lsstccs/eo_bright_defects_13550_w_2023_41/20231117T000800Z', id=481a83cb-d52e-42ca-9d47-7403d3cb00b2)]

In [7]:
refList = rsu.generateRefList(repo,collections,13550,"raw",55,"DARK")
refList

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fd5df11df50>>
Traceback (most recent call last):
  File "/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [8]:
detector_num = 100
iteratr = 0
kwargs={
        "datasetType": "",
        "collections": "",
        "where":"instrument='LSSTCam' and detector = {var}".format(var=detector_num)
        }
defect_path = ("eo_bright_defects","eo_dark_defects")[iteratr]
run = 13550
week = 'w_2023_41'
kwargs['datasetType'] = ("eoBrightDefects","eoDarkDefects")[iteratr]
kwargs['collections'] = "u/lsstccs/"+defect_path+"_"+str(run)+"_"+week

reg = rsu.makeRegistry(rsu.makeButler(repo,collections))
dsets = list(reg.queryDatasets(**kwargs))
mask = rsu.makeButler(repo,collections).get(dsets[0])

In [9]:
kwargs,{'datasetType': 'eoBrightDefects', 'collections': 'u/lsstccs/eo_bright_defects_13550_w_2023_41', 'where': "exposure.science_program in ('13550') and instrument='LSSTCam' and detector=55"}

({'datasetType': 'eoBrightDefects',
  'collections': 'u/lsstccs/eo_bright_defects_13550_w_2023_41',
  'where': "instrument='LSSTCam' and detector = 100"},
 {'datasetType': 'eoBrightDefects',
  'collections': 'u/lsstccs/eo_bright_defects_13550_w_2023_41',
  'where': "exposure.science_program in ('13550') and instrument='LSSTCam' and detector=55"})

In [10]:
mask.toFitsRegionTable().__dir__()

['_masked',
 '_column_class',
 'columns',
 'formatter',
 '_copy_indices',
 '_init_indices',
 'primary_key',
 '_meta',
 '__module__',
 '__doc__',
 'meta',
 'Row',
 'Column',
 'MaskedColumn',
 'TableColumns',
 'TableFormatter',
 'read',
 'write',
 'pprint_exclude_names',
 'pprint_include_names',
 'as_array',
 '__init__',
 '_set_column_attribute',
 '__getstate__',
 '__setstate__',
 'mask',
 '_mask',
 'filled',
 'indices',
 'loc',
 'loc_indices',
 'iloc',
 'add_index',
 'remove_indices',
 'index_mode',
 '__array__',
 '_check_names_dtype',
 '_init_from_list_of_dicts',
 '_init_from_list',
 '_convert_data_to_col',
 '_init_from_ndarray',
 '_init_from_dict',
 '_get_col_cls_for_table',
 '_convert_col_for_table',
 '_init_from_cols',
 '_new_from_slice',
 '_make_table_from_cols',
 '_set_col_parent_table_and_mask',
 'itercols',
 '_base_repr_',
 '_repr_html_',
 '__repr__',
 '__str__',
 '__bytes__',
 'has_mixin_columns',
 'has_masked_columns',
 'has_masked_values',
 '_is_mixin_for_table',
 'pprint',
 

In [21]:
mask.toTable()

[<Table length=1>
   x0    y0  width height
 int64 int64 int64 int64 
 ----- ----- ----- ------
  3931  1347     1      1]

In [122]:
collection1 = 'u/lsstccs/eo_bright_defects_13550_w_2023_41'
collection2 = 'u/lsstccs/eo_bright_defects_13401_w_2023_24'
collection3 = 'u/lsstccs/eo_dark_current_13401_w_2023_24'


In [123]:
msk1 = rsu.getDefectMask(repo,collection1,[13550],"eoBrightDefects",55)
msk2 = rsu.getDefectMask(repo,collection2,[13401],"eoBrightDefects",55)


In [49]:
# msk2.toFitsRegionTable() - msk1.toFitsRegionTable()

In [51]:
np.unique(msk1.toFitsRegionTable()["SHAPE"])

BOX
POINT


In [53]:
type(msk1.toFitsRegionTable())

astropy.table.table.Table

In [67]:
reg = rsu.makeRegistry(rsu.makeButler(repo,collections))

In [69]:
list(reg.queryDatasets(...,collections=collections).byParentDatasetType())

[<DatasetRef iterator for [components of] camera>,
 <DatasetRef iterator for [components of] raw>,
 <DatasetRef iterator for [components of] cpBiasProc>,
 <DatasetRef iterator for [components of] isr_metadata>,
 <DatasetRef iterator for [components of] bias>,
 <DatasetRef iterator for [components of] cpCombine_metadata>,
 <DatasetRef iterator for [components of] isr_config>,
 <DatasetRef iterator for [components of] cpCombine_config>,
 <DatasetRef iterator for [components of] packages>,
 <DatasetRef iterator for [components of] cpCrosstalkProc>,
 <DatasetRef iterator for [components of] crosstalkBackgroundZOffsets>,
 <DatasetRef iterator for [components of] crosstalkBackgroundYTilts>,
 <DatasetRef iterator for [components of] crosstalkRatios>,
 <DatasetRef iterator for [components of] crosstalkBackgroundXTilts>,
 <DatasetRef iterator for [components of] crosstalkRatioErrors>,
 <DatasetRef iterator for [components of] crosstalkFluxes>,
 <DatasetRef iterator for [components of] crosstalk

In [141]:
dark_collection = reg.queryCollections(f"*eo_dark_current_13401*",collectionTypes=daf_butler.CollectionType.CHAINED)
return_dict = eo_pipe.get_amp_data(repo,dark_collection)
dark_current = return_dict["dark_current_median"]["R23_S10"]["C10"]

0.008040801621973515

In [136]:
returns.keys()

dict_keys(['read_noise', 'bias_level', 'dark_current_95', 'dark_current_median', 'divisadero_tearing', 'ptc_a00', 'ptc_gain', 'ptc_noise', 'ptc_turnoff'])

In [129]:
dark_collection 

['u/lsstccs/eo_dark_current_13401_w_2023_24']

In [118]:
myKwargs

{'datasetType': 'eoDarkCurrent',
 'collections': 'u/lsstccs/eo_dark_current_13401',
 'where': "instrument='LSSTCam' and detector=100"}

In [117]:
myKwargs = rsu.makeKwargs([13401],"eoDarkCurrent",100,dark_collection)
myKwargs["where"] = " ".join(myKwargs["where"].split(" ")[4:])
reg = rsu.makeRegistry(rsu.makeButler(repo,collection3))

rsu.getRefList(reg,myKwargs)

MissingCollectionError: No collection with name 'u/lsstccs/eo_dark_current_13401' found.

('/repo/ir2', 'u/lsstccs/eo_dark_current_13401_w_2023_24')

In [79]:

obj3 = rsu.fetchAllRefs(repo,collections,[13401],"eoDarkCurrent",55)


ColumnError: Cannot join dimension element exposure whose dimensions are not a subset of {instrument, detector}.